In [13]:
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import json

In [14]:
WRITE_IMAGES = False
INIT = "t"
SAVE_PATH = "../results_cmp_rewards_{}.{}"

In [15]:
# models = {
#     "b_dist-lstm-v-61": {"path" : f"../results-nav-b_dist-lstm-v-61/plt/i{INIT}/", "color": "purple"},
#     # "dist-err-rew-wi-v-50": {"path" : f"../results-nav-dist-err-rew-w_i-v-50/plt/i{INIT}/", "color": "red"},
#     "b_dist-err-rew-wi-v-51": {"path" : f"../results-nav-b_dist-err-rew-w_i-v-51/plt/i{INIT}/", "color": "orange"},
#     # "dist-v-3": {"path" : f"../results-nav-dist-v-3/plt/i{INIT}/", "color": "blue"},
#     "b_dist-v-31": {"path" : f"../results-nav-b_dist-v-31/plt/i{INIT}/", "color": "blue"},
#     "v-21": {"path" : f"../results-nav-v-21/plt/i{INIT}/", "color": "green"},
# }

models = {
    "lstm": {"path" : f"../nav-results4-lstm/plt/i{INIT}/", "color": "purple"},
    "dist-err": {"path" : f"../nav-results3-dist-err/plt/i{INIT}/", "color": "orange"},
    "dist-err-u": {"path" : f"../nav-results3-dist-err-u/plt/i{INIT}/", "color": "black"},
    "dist": {"path" : f"../nav-results2-dist/plt/i{INIT}/", "color": "blue"},
    # "dist-v0": {"path" : f"../nav-results2-dist-v0/plt/i{INIT}/", "color": "black"},
    "baseline": {"path" : f"../nav-results1-baseline/plt/i{INIT}/", "color": "green"},
}

In [21]:
dist_dirs = ["x", "z", "xyz"]

for dir in dist_dirs:
    fig = px.line()
    fig.update_layout(
        title_text=f"Disturbance Direction: {dir}",
        title_xanchor="center",
        title_x=0.5,
        xaxis_title="Magnitude of Disturbace",
        yaxis_title="Expected Reward",
    )

    for mdl in models:
        df = pd.read_csv(os.path.join(models[mdl]["path"], "ext_dists_res_df.csv"))
        df["dist_mag"] = df.apply(lambda row: np.sum(list(map(float, list(row["dist"][1:-1].split())))), axis=1)

        temp_df = df[df.dir == dir]
        temp_df = temp_df.sort_values(by=["dist_mag"])

        fig.add_scatter(
            x=temp_df.dist_mag,
            y=temp_df.mean_step_reward,
            line=dict(color=models[mdl]["color"]),
            name=mdl
        )


    fig.update_layout(width=800, height=400)
    fig.update_yaxes(range=[-3, 0])
    fig.show()

    if WRITE_IMAGES:
        fig.write_image(SAVE_PATH.format(dir, "png"))
        fig.write_html(SAVE_PATH.format(dir, "html"))
